In [1]:
import scanpy as sc
import numpy as np
adata = sc.read('../data/SATURN_alignment.h5ad')
adata.X = np.ceil(adata.X)
# adata.X = (adata.X > 0).astype(np.int32)
# print(adata)
# sc.pp.normalize_total(adata, target_sum=1e4)
# sc.pp.log1p(adata)
print(adata.X)

[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


/Users/jiayidan/Library/Python/3.9/lib/python/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [ ]:
adata = sc.read('data/SATURN_alignment.h5ad')

array([[0.       , 0.       , 0.       , ..., 6.0346837, 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 6.0771437, 0.       ,
        0.       ],
       ...,
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ]], dtype=float32)

In [33]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

frog_labels = adata.obs.loc[adata.obs['species'] == 'frog', 'ref_labels'].unique()
zebrafish_labels = adata.obs.loc[adata.obs['species'] == 'zebrafish', 'ref_labels'].unique()
intersection_labels = sorted(list(set(frog_labels).intersection(set(zebrafish_labels))))
print("Intersection label count:", len(intersection_labels))
print("Labels:", intersection_labels)

adata_filt = adata[adata.obs['ref_labels'].isin(intersection_labels)].copy()
label_encoder = LabelEncoder()
adata_filt.obs['ref_labels_enc'] = label_encoder.fit_transform(adata_filt.obs['ref_labels'])
label_map = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
label_map = {k: int(v) for k, v in label_map.items()}
print("Label mapping:", label_map)

adata_frog = adata_filt[adata_filt.obs['species'] == 'frog']
adata_zeb = adata_filt[adata_filt.obs['species'] == 'zebrafish']
X_s = adata_frog.X.toarray() if hasattr(adata_frog.X, "toarray") else np.array(adata_frog.X)
y_s = adata_frog.obs['ref_labels_enc'].values
X_t = adata_zeb.X.toarray() if hasattr(adata_zeb.X, "toarray") else np.array(adata_zeb.X)
y_t = adata_zeb.obs['ref_labels_enc'].values

print("X_s:", X_s.shape, "y_s:", y_s.shape)
print("X_t:", X_t.shape, "y_t:", y_t.shape)


Intersection label count: 28
Labels: ['Blood', 'Endoderm', 'Endothelial', 'Epidermal progenitor', 'Eye primordium', 'Forebrain/midbrain', 'Germline', 'Hatching gland', 'Heart', 'Hindbrain', 'Intermediate mesoderm', 'Involuting marginal zone', 'Ionocyte', 'Lens', 'Neural crest', 'Neuroectoderm', 'Neuron', 'Non-neural ectoderm', 'Notochord', 'Notoplate', 'Olfactory placode', 'Optic', 'Otic placode', 'Placodal area', 'Presomitic mesoderm', 'Rohon-beard neuron', 'Skeletal muscle', 'Tailbud']
Label mapping: {'Blood': 0, 'Endoderm': 1, 'Endothelial': 2, 'Epidermal progenitor': 3, 'Eye primordium': 4, 'Forebrain/midbrain': 5, 'Germline': 6, 'Hatching gland': 7, 'Heart': 8, 'Hindbrain': 9, 'Intermediate mesoderm': 10, 'Involuting marginal zone': 11, 'Ionocyte': 12, 'Lens': 13, 'Neural crest': 14, 'Neuroectoderm': 15, 'Neuron': 16, 'Non-neural ectoderm': 17, 'Notochord': 18, 'Notoplate': 19, 'Olfactory placode': 20, 'Optic': 21, 'Otic placode': 22, 'Placodal area': 23, 'Presomitic mesoderm': 24

/Users/jiayidan/Library/Python/3.9/lib/python/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [34]:
np.savez(
    "data/frog_zeb_data_processed.npz",
    X_s=X_s,
    y_s=y_s,
    X_t=X_t,
    y_t=y_t,
    label_map=label_map
)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import scanpy as sc

# 读取数据
adata = sc.read('../data/SATURN_alignment.h5ad')

# ================
# 1. 找共同 label
# ================
frog_labels = adata.obs.loc[adata.obs['species'] == 'frog', 'ref_labels'].unique()
zebrafish_labels = adata.obs.loc[adata.obs['species'] == 'zebrafish', 'ref_labels'].unique()
intersection_labels = sorted(list(set(frog_labels).intersection(set(zebrafish_labels))))
print("Intersection label count:", len(intersection_labels))
print("Labels:", intersection_labels)


# 过滤只保留共同 label 的细胞
adata_filt = adata[adata.obs['ref_labels'].isin(intersection_labels)].copy()

# ================
# 2. 编码 ref_labels
# ================
label_encoder = LabelEncoder()
adata_filt.obs['ref_labels_enc'] = label_encoder.fit_transform(adata_filt.obs['ref_labels'])
adata_filt.write("../adata_filt.h5ad")
label_map = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
label_map = {k: int(v) for k, v in label_map.items()}
print("Label mapping:", label_map)

# ================
# 3. 对 adata_filt.X 做 PCA → 50维
#   （使用 scanpy 内置 PCA）
# ================
sc.tl.pca(adata_filt, n_comps=100, use_highly_variable=False)

# PCA 结果保存在 adata_filt.obsm["X_pca"]
X_pca = adata_filt.obsm["X_pca"]
print("PCA done. Shape:", X_pca.shape)   # (N_cells, 50)

# ================
# 4. 按 species 拆回 source / target
# ================
is_frog = adata_filt.obs['species'] == 'frog'
is_zeb  = adata_filt.obs['species'] == 'zebrafish'

X_s = X_pca[is_frog.values]
y_s = adata_filt.obs.loc[is_frog, 'ref_labels_enc'].values

X_t = X_pca[is_zeb.values]
y_t = adata_filt.obs.loc[is_zeb, 'ref_labels_enc'].values

# 打印维度
print("X_s:", X_s.shape, "y_s:", y_s.shape)
print("X_t:", X_t.shape, "y_t:", y_t.shape)

# ================
# 5. 可选：保存
# ================
np.savez("../data/frog_zeb_pca100.npz",
         X_s=X_s,
         y_s=y_s,
         X_t=X_t,
         y_t=y_t,
         label_map=np.array(list(label_map.items()), dtype=object))

print("Saved to data/frog_zeb_pca100.npz")


/Users/jiayidan/Library/Python/3.9/lib/python/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


Intersection label count: 28
Labels: ['Blood', 'Endoderm', 'Endothelial', 'Epidermal progenitor', 'Eye primordium', 'Forebrain/midbrain', 'Germline', 'Hatching gland', 'Heart', 'Hindbrain', 'Intermediate mesoderm', 'Involuting marginal zone', 'Ionocyte', 'Lens', 'Neural crest', 'Neuroectoderm', 'Neuron', 'Non-neural ectoderm', 'Notochord', 'Notoplate', 'Olfactory placode', 'Optic', 'Otic placode', 'Placodal area', 'Presomitic mesoderm', 'Rohon-beard neuron', 'Skeletal muscle', 'Tailbud']


/Users/jiayidan/Library/Python/3.9/lib/python/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


Label mapping: {'Blood': 0, 'Endoderm': 1, 'Endothelial': 2, 'Epidermal progenitor': 3, 'Eye primordium': 4, 'Forebrain/midbrain': 5, 'Germline': 6, 'Hatching gland': 7, 'Heart': 8, 'Hindbrain': 9, 'Intermediate mesoderm': 10, 'Involuting marginal zone': 11, 'Ionocyte': 12, 'Lens': 13, 'Neural crest': 14, 'Neuroectoderm': 15, 'Neuron': 16, 'Non-neural ectoderm': 17, 'Notochord': 18, 'Notoplate': 19, 'Olfactory placode': 20, 'Optic': 21, 'Otic placode': 22, 'Placodal area': 23, 'Presomitic mesoderm': 24, 'Rohon-beard neuron': 25, 'Skeletal muscle': 26, 'Tailbud': 27}


/Users/jiayidan/Library/Python/3.9/lib/python/site-packages/scanpy/preprocessing/_pca.py:377: FutureWarning: Argument `use_highly_variable` is deprecated, consider using the mask argument. Use_highly_variable=True can be called through mask_var="highly_variable". Use_highly_variable=False can be called through mask_var=None
  warn(msg, FutureWarning)


PCA done. Shape: (146669, 100)
X_s: (89702, 100) y_s: (89702,)
X_t: (56967, 100) y_t: (56967,)
Saved to data/frog_zeb_pca100.npz
